# Customer Service Chatbot Application

## Import liberaries and conversation file

In [1]:
import pandas as pd
import numpy as np
import gradio as gr

# utils...
import json
import requests
import random
import time

import spacy #spacy for quick language prepro
nlp = spacy.load('da_core_news_md') #instantiating Danish module

from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model

# For controlling warnings in the code
import warnings
from sklearn.exceptions import ConvergenceWarning

/Users/tobiasmjensen/anaconda3/envs/bds-chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# We will call the below code to ignore all selected warnings in the code for getting more clearn outputs
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

A JSON file, like "convo.json," can be useful for organizing conversation flows, storing responses, or defining rules for a customer service chatbot. It's a structured way to manage dialogue trees, intents, entities, and responses within a chatbot framework. However, whether you specifically need a file named "convo.json" depends on the framework or platform you're using to build your chatbot.

Many chatbot development platforms or frameworks offer their own ways to structure conversations, sometimes using JSON or similar formats. These files help maintain and organize the chatbot's knowledge base, making it easier to update and manage conversations.

If you're developing a chatbot from scratch or using a platform that requires such a file, creating a "convo.json" or a similar structured file for your customer service chatbot could be beneficial. It allows you to define the chatbot's responses, conversation paths, and logic in a clear, organized manner.

In [3]:
filepath = '/Users/tobiasmjensen/Documents/aau_bds/M3 - Semester Project/M3_Semester_Project/convo_snt_chatbot.json'
data = json.load(open(filepath,'r'))

# Data Preprocessing

In [4]:
def text_prepro(texts):
  """
  takes in a list/iterable of texts
  lowercases, normalizes text
  """

  clean_container = []

  for text in nlp.pipe(texts, disable=["parser", "ner"]):

    txt = [token.lemma_.lower() for token in text # lemmatize and lower
          if token.is_alpha # remove numbers
          and not token.is_punct] # remove punctoation

    clean_container.append(" ".join(txt))

  return clean_container

In [5]:
training = []

for i in data['intents']:
  l = len(i['patterns'])
  tuples = list(zip(i['patterns'], l*[i['tag']]))
  training.extend(tuples)

In [6]:
training_df = pd.DataFrame(training, columns=['txt','label'])
training_df.head()

,txt,label
0,Hej,hilsen
1,Halløj,hilsen
2,Goddag,hilsen
3,God dag,hilsen
4,Går det godt?,svar


In [7]:
training_df.txt_p = text_prepro(training_df.txt)

# Model loading

In [ ]:
bot_name = '💬 SNT Bot'

## TfidfVectorizer

In [ ]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
cls = LogisticRegression()

tfidf_pipe = make_pipeline(tfidf, cls)
tfidf_pipe.fit(training_df.txt_p, training_df.label)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [ ]:
tfidf_pipe.predict(['Hvordan indtaster jeg rabatkode?'])

array(['Sådan indtaster du din rabatkode eller dit gavekort'],
      dtype=object)

In [ ]:
def tfidf_reply(txt):
  clean_text = text_prepro([txt])
  tag = tfidf_pipe.predict(clean_text)[0]
  for intent in data['intents']:
    if tag == intent["tag"]:
      return f"{bot_name}: {random.choice(intent['responses'])}"

In [ ]:
tfidf_reply('Hvordan bruger jeg en rabatkode?')

"💬 SNT Bot: Rabatkoder og gavekort indtastes i samme felt. Følg de få trin herunder for at finde feltet:\n\n1. Når du har valgt det ønskede produkt, trykker du på indkøbskurven øverst i højre hjørne\n2. Acceptér så handelsbetingelserne og tryk 'Gå til betaling'\n3. Herefter kommer du til siden 'Oplysninger', hvor du skal klikke på 'Vis ordreoversigt'\n4. Du kan nu indtaste dit gavekort eller din rabatkode i feltet\n\nHvis du er i besiddelse af et fysisk gavekort i form af et kreditkort, kan du bruge dette online og i de danske butikker ved at indtaste cifrene undtagen de første fem.\n\nBemærk venligst: Det ikke er muligt at tilføje en rabatkode til en ordre, hvis man bestiller noget, der i forvejen er nedsat (sæt- eller kvantumrabat)."

In [ ]:
tfidf_reply('Kan jeg passe bukserne hvis jeg har store lår?')

'💬 SNT Bot: Hvis du har store lår eller lægge plejer det, som udgangspunkt, ikke at være noget problem. Vi tilbyder nemlig både Slim og Regular fit.\n\nDu kan altid prøve vores bukser på i nogle timer og derefter bestemme dig for, om du synes, at de har givet sig tilstrækkeligt og sidder, som du ønsker det.'

## Word2Vec !!HELP!!

In [50]:
from gensim.models import Word2Vec

# Train your Word2Vec model
corpus = [...]  # Your text corpus
word2vec_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=5, workers=4)

TypeError: 'ellipsis' object is not iterable

In [42]:
#instantiate models and "bundle up as pipeline"

w2v = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
cls = LogisticRegression()


w2v_pipe = make_pipeline(w2v, cls)
w2v_pipe.fit(training_df.txt_p, training_df.label)

In [43]:
w2v_pipe = make_pipeline(w2v, cls)
w2v_pipe.fit(training_df.txt_p, training_df.label)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'Word2Vec<vocab=12, vector_size=100, alpha=0.025>' (type <class 'gensim.models.word2vec.Word2Vec'>) doesn't

In [9]:
w2v_pipe.predict(['Hvordan indtaster jeg rabatkode?'])

array(['Sådan indtaster du din rabatkode eller dit gavekort'],
      dtype=object)

In [29]:
def w2v_reply(txt):
  clean_text = text_prepro([txt])
  tag = w2v_pipe.predict(clean_text)[0]
  for intent in data['intents']:
    if tag == intent["tag"]:
      return f"{bot_name}: {random.choice(intent['responses'])}"

In [30]:
w2v_reply('Hvordan bruger jeg en rabatkode?')

"💬 SNT Bot: Rabatkoder og gavekort indtastes i samme felt. Følg de få trin herunder for at finde feltet:\n\n1. Når du har valgt det ønskede produkt, trykker du på indkøbskurven øverst i højre hjørne\n2. Acceptér så handelsbetingelserne og tryk 'Gå til betaling'\n3. Herefter kommer du til siden 'Oplysninger', hvor du skal klikke på 'Vis ordreoversigt'\n4. Du kan nu indtaste dit gavekort eller din rabatkode i feltet\n\nHvis du er i besiddelse af et fysisk gavekort i form af et kreditkort, kan du bruge dette online og i de danske butikker ved at indtaste cifrene undtagen de første fem.\n\nBemærk venligst: Det ikke er muligt at tilføje en rabatkode til en ordre, hvis man bestiller noget, der i forvejen er nedsat (sæt- eller kvantumrabat)."

In [31]:
w2v_reply('Kan jeg passe bukserne hvis jeg har store lår?')

'💬 SNT Bot: Hvis du har store lår eller lægge plejer det, som udgangspunkt, ikke at være noget problem. Vi tilbyder nemlig både Slim og Regular fit.\n\nDu kan altid prøve vores bukser på i nogle timer og derefter bestemme dig for, om du synes, at de har givet sig tilstrækkeligt og sidder, som du ønsker det.'

# Application

In [32]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        # use the reply_da function to get the bot's message
        bot_message = tfidf_reply(message)
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
